In [1]:
import os
import json
import random
from sklearn.model_selection import train_test_split
from collections import Counter
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import shuffle
import numpy as np
import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

In [2]:
initial_feature_names = [
    'targetName',
    'className',
    'returnType',
    'methodModifiers',
    'otherAnnotations',
    'otherMethodsNames',
    'otherMethodsAnnotations',
    'returnsNull',
    'checksNull',
    'targetType'
]
class AnnotationUsage:
    def __init__(self, usage_json):
        self.annotation_name = usage_json['name']
        features_json = usage_json['features']
        self.features_list = [
            features_json.get('targetName', ''),
            features_json.get('className', ''),
            features_json.get('returnType', ''),
            features_json.get('methodModifiers', []),
            features_json.get('otherAnnotations', []),
            features_json.get('otherMethodsNames', []),
            features_json.get('otherMethodsAnnotations', []),
            1 if features_json.get('returnsNull', False) else 0,
            1 if features_json.get('checksNull', False) else 0,
        ]
        self.file_path = usage_json['filePath']

    def __str__(self):
        return f'{self.annotation_name}'

class UsagesLoader:
    def __init__(self, processing_result_path):
        self.processing_result_path = processing_result_path

    def load(self):
        usages_by_target = defaultdict(list)
        for root, dirs, files in os.walk(self.processing_result_path):
            for file in files:
                if not file.endswith('json'):
                    continue
                with open(os.path.join(root, file), 'r') as read_file:
                    data = json.load(read_file)
                    target_type = data['keyInfo']['name']
                    new_usages = [AnnotationUsage(usage_json) for usage_json in data["usages"]]
                    for usage in new_usages:
                        usage.features_list.append(target_type)
                    usages_by_target[target_type] = usages_by_target[target_type] + new_usages
        return usages_by_target

In [3]:
class Baseline:
    def __init__(self):
        self.ordered_by_quantity = np.array([])

    def fit(self, X, y):
        self.ordered_by_quantity = np.array([x[0] for x in Counter(y).most_common()])

    def predict(self, X):
        return np.array([self.ordered_by_quantity for _ in X])

In [4]:
class Metric:
    def __init__(self, predicted_y, expected_y):
        self.orders = [(list(predicted_y[i]) + [expected_y[i]]).index(expected_y[i]) + 1 for i in range(len(expected_y))]
        # for i in range(len(expected_y)):
        #     if expected_y[i] != predicted_y[i][0]:
        #         print(expected_y[i], predicted_y[i][0])

    def top_i(self, i):
        return sum(map(lambda x: x <= i, self.orders)) / len(self.orders)

In [92]:
def encode_names(column, train_size, column_name, max_new_columns):
    """
    Converts column of camelCase names to 100 columns with most popular words
    with 1 (if name contains word) and 0 (otherwise)
    """

    def split_camel_case(x):
        words = [word.lower() for word in re.findall(r'[A-Z]?[a-z]+|[A-Z]+(?=[A-Z]|$)', x)]
        return ' '.join(words)

    vectorizer = CountVectorizer(preprocessor=split_camel_case, max_features=max_new_columns)
    vectorizer.fit(column[:train_size])
    new_columns = vectorizer.transform(column).toarray()
    new_names = [column_name + '_' + name for name in vectorizer.get_feature_names_out()]
    return new_columns, new_names

def encode_lists(column, train_size, column_name, max_new_columns):
    """
    Converts column of lists of words to 100 columns with most popular words
    with 1 (if list contains word) and 0 (otherwise)
    """

    joined_words = np.array([' '.join(x).replace('.', '') for x in column])

    vectorizer = CountVectorizer(max_features=max_new_columns)
    vectorizer.fit(joined_words[:train_size])
    new_columns = vectorizer.transform(joined_words).toarray()
    new_names = [column_name + '_' + name for name in vectorizer.get_feature_names_out()]
    return new_columns, new_names


def encode_column(column, train_size, column_name, max_new_columns):
    """
    Converts column of some type to column of integers
    """

    if isinstance(column[0], str):
        if (column[:train_size] == '').all():
            return None, []
        return encode_names(column, train_size, column_name, max_new_columns)
    elif isinstance(column[0], list):
        if np.alltrue([x == [] for x in column[:train_size]]):
            return None, []
        return encode_lists(column, train_size, column_name, max_new_columns)
    else:
        return np.array([column.astype(np.int64)]).T, [column_name]

In [93]:
encode_column(np.array(['getSomethingGood', 'returnSomething'], dtype=object), 1, 'a', 100)

(array([[1, 1, 1],
        [0, 0, 1]]),
 ['a_get', 'a_good', 'a_something'])

In [94]:
encode_column(np.array([['public'], ['public', 'static']], dtype=object), 1, 'b', 100)

(array([[1],
        [1]]),
 ['b_public'])

In [95]:
encode_column(np.array([1, 0], dtype=object), 1, 'c', 100)

(array([[1],
        [0]]),
 ['c'])

In [96]:
usages_loader = UsagesLoader(
    '/Users/danilbk/Programming/Kotlin/test/intellij-community/project-processing-results/processing/java/annotations/processing/0.0.0')
usages_by_target_type = usages_loader.load()

In [97]:
def load_for_target(target_type, max_new_columns = 100, size = 10000, train_size = 0.8, state = 42, ignored_annotation=()):
    method_usages = list(
        filter(lambda x: x.annotation_name not in ignored_annotation, usages_by_target_type[target_type]))
    method_usages = shuffle(method_usages, random_state=state)[:size]
    raw_X = np.array([np.array(usage.features_list, dtype=object) for usage in method_usages])
    X = None
    all_new_names = []
    if len(raw_X) == 0:
        X = np.array([])
        all_new_names = initial_feature_names
    else:
        for col in range(raw_X.shape[1]):
            new_columns, new_names = encode_column(raw_X[:, col], round(len(raw_X[:, col]) * train_size), initial_feature_names[col], max_new_columns)
            if new_columns is None:
                continue
            all_new_names += new_names
            if X is None:
                X = new_columns
            else:
                X = np.concatenate((X, new_columns), axis=1)
    y = np.array([usage.annotation_name for usage in method_usages])
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, shuffle=False)
    return X_train, X_test, y_train, y_test, all_new_names

In [35]:
def calculate(X_train, X_test, y_train, y_test, model):
    # X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.2, shuffle=True, random_state=42)

    model.fit(X_train, y_train)
    predicted = model.predict(X_test)

    metric = Metric(predicted, y_test)
    # print(f'Count: {len(y_test)}')
    for i in range(1, 2):
        print(f'Top {i}: {metric.top_i(i)}')

In [13]:
target_types = [
    'AnnotationType',
    'Constructor',
    'Field',
    'LocalVariable',
    'Method',
    'Module',
    'Package',
    'Parameter',
    'RecordComponent',
    'Type',
    'TypeParameter',
    'TypeUse'
]

In [14]:
class WrapperModel:
    def __init__(self, model):
        self.model = model

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return np.array([np.flip(self.model.classes_[np.argsort(proba)]) for proba in self.model.predict_proba(X)])


In [15]:
from sklearn.svm import SVC

class SVM:
    def __init__(self, c=1, need_scaler=False):
        self.svm = SVC(C=c, probability=True)
        self.model = self.svm
        if need_scaler:
            self.model = make_pipeline(StandardScaler(), self.model)
        # self.model = SVC(kernel=kernel, probability=True, degree=degree)

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return np.array([np.flip(self.model.classes_[np.argsort(proba)]) for proba in self.model.predict_proba(X)])
        # return np.array([np.array([y_pred]) for y_pred in self.model.predict(X)])

In [49]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

class LinearSVM:
    def __init__(self, c=1, need_scaler=False):
        self.model = LinearSVC(C=c, max_iter=1000)
        if need_scaler:
            self.model = make_pipeline(StandardScaler(), self.model)
        self.model = CalibratedClassifierCV(self.model)
        # self.model = OneVsRestClassifier(self.model)

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return np.array([np.flip(self.model.classes_[np.argsort(proba)]) for proba in self.model.predict_proba(X)])
        # return np.array([np.array([y_pred]) for y_pred in self.model.predict(X)])

In [17]:
from sklearn.linear_model import LogisticRegression

class LogisticReg:
    def __init__(self, C=1, solver='lbfgs', need_scaler=False):
        self.model = LogisticRegression(max_iter=1000, C=C, solver=solver)
        if need_scaler:
            self.model = make_pipeline(StandardScaler(), self.model)

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return np.array([np.flip(self.model.classes_[np.argsort(proba)]) for proba in self.model.predict_proba(X)])
        # return np.array([np.array([y_pred]) for y_pred in self.model.predict(X)])
    def score(self, X, y=None, sample_weight=None):
        self.model.score(X, y=y, sample_weight=sample_weight)

In [18]:
from sklearn.ensemble import RandomForestClassifier

class RandomForest:
    def __init__(self):
        self.model = RandomForestClassifier(n_estimators=100, n_jobs=8)
        # self.model = SVC(kernel=kernel, probability=True, degree=degree)

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return np.array([np.flip(self.model.classes_[np.argsort(proba)]) for proba in self.model.predict_proba(X)])
        # return np.array([np.array([y_pred]) for y_pred in self.model.predict(X)])

In [19]:
from sklearn.ensemble import GradientBoostingClassifier

class GradientBoosting:
    def __init__(self):
        self.model = GradientBoostingClassifier()
        # self.model = SVC(kernel=kernel, probability=True, degree=degree)

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return np.array([np.flip(self.model.classes_[np.argsort(proba)]) for proba in self.model.predict_proba(X)])
        # return np.array([np.array([y_pred]) for y_pred in self.model.predict(X)])

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

class AdaBoost:
    def __init__(self):
        base_estim = DecisionTreeClassifier(max_depth=1, max_features='sqrt')
        self.model = AdaBoostClassifier(base_estimator=base_estim, n_estimators=500, learning_rate=0.5)
        # self.model = SVC(kernel=kernel, probability=True, degree=degree)

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return np.array([np.flip(self.model.classes_[np.argsort(proba)]) for proba in self.model.predict_proba(X)])
        # return np.array([np.array([y_pred]) for y_pred in self.model.predict(X)])

In [21]:
from catboost import CatBoostClassifier

class CatBoost:
    def __init__(self):
        self.model = CatBoostClassifier(
                                        verbose=False,
                                        thread_count=8)
        # self.model = SVC(kernel=kernel, probability=True, degree=degree)

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return np.array([np.flip(self.model.classes_[np.argsort(proba)]) for proba in self.model.predict_proba(X)])
        # return np.array([np.array([y_pred]) for y_pred in self.model.predict(X)])

In [22]:
from sklearn.linear_model import RidgeClassifierCV

class Ridge:
    def __init__(self, c=1, need_scaler=False):
        if need_scaler:
            self.model = make_pipeline(StandardScaler(), RidgeClassifierCV())
        else:
            self.model = RidgeClassifierCV()

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return np.array([np.flip(self.model.classes_[np.argsort(proba)]) for proba in self.model.predict_proba(X)])
        # return np.array([np.array([y_pred]) for y_pred in self.model.predict(X)])

In [23]:
from sklearn.neural_network import MLPClassifier

class MLP:
    def __init__(self):
        self.model = MLPClassifier(verbose=False, hidden_layer_sizes=(10, 10, 10))

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return np.array([np.flip(self.model.classes_[np.argsort(proba)]) for proba in self.model.predict_proba(X)])
        # return np.array([np.array([y_pred]) for y_pred in self.model.predict(X)])

In [ ]:
for degree in [3, 4, 5, 10, 20, 100]:
    for target_type in ['Method']:
        X, y, feature_names = load_for_target(target_type)
        rnd = random.Random(42)
        for _ in range(1):
            size = 10000
            X, y = shuffle(X, y)
            X = X[:size]
            y = y[:size]
            if len(X) == 0:
                continue
            print(degree)
            svm = SVM()
            print("SVM")
            calculate(X, y, svm)
            print("Logistic regression")
            calculate(X, y, LogisticReg())
            print("Baseline")
            calculate(X, y, Baseline())
            print()

In [98]:
for target_type in ['Method']:
    X_train, X_test, y_train, y_test, feature_names = load_for_target(target_type, 100, size=10000, train_size=0.8)
    for _ in range(1):
        if len(X) == 0:
            continue
        print(target_type)
        print("Linear SVM")
        calculate(X_train, X_test, y_train, y_test, LinearSVM())
        print("CatBoost")
        calculate(X_train, X_test, y_train, y_test, CatBoost())
        print("Logistic regression")
        calculate(X_train, X_test, y_train, y_test, LogisticReg())
        # model = GridSearchCV(estimator=LogisticRegression(),
        #                      param_grid={'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 'C': [0.01, 0.1, 0.2, 1, 2, 10]})
        # calculate(X, y, WrapperModel(model))
        print("Baseline")
        calculate(X_train, X_test, y_train, y_test, Baseline())
        print()

Method
Linear SVM


/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/svm/_base.py:

Top 1: 0.874
CatBoost



KeyboardInterrupt



In [45]:
for target_type in target_types:
    for _ in range(1):
        X, y, feature_names = load_for_target(target_type)
        size = 10000
        X, y = shuffle(X, y)
        X = X[:size]
        y = y[:size]
        if len(X) == 0:
            continue
        print(target_type)
        print("CatBoost")
        calculate(X, y, CatBoost())
        print("Gradient Boosting")
        calculate(X, y, GradientBoosting())
        print("Random Forest")
        calculate(X, y, RandomForest())
        print("SVM")
        calculate(X, y, SVM())
        print("Linear SVM")
        calculate(X, y, LinearSVM())
        print("Logistic regression")
        calculate(X, y, LogisticReg())
        print("Baseline")
        calculate(X, y, Baseline())
        print()

AnnotationType
CatBoost


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Top 1: 0.8977635782747604
Gradient Boosting
Top 1: 0.8498402555910544
Random Forest
Top 1: 0.8785942492012779
SVM
Top 1: 0.8210862619808307
Linear SVM
Top 1: 0.8785942492012779
Logistic regression


/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Top 1: 0.8594249201277955
Baseline
Top 1: 0.38338658146964855

Constructor
CatBoost
Top 1: 0.5237226277372263
Gradient Boosting
Top 1: 0.44525547445255476
Random Forest
Top 1: 0.4981751824817518
SVM
Top 1: 0.4197080291970803
Linear SVM


...

Top 1: 0.837375
Logistic regression
Top 1: 0.82675
Baseline
Top 1: 0.836375

Type
CatBoost
Top 1: 0.8151983860121049
Gradient Boosting
Top 1: 0.8111634162743779
Random Forest
Top 1: 0.8045729657027573
SVM
Top 1: 0.7339609952925353
Linear SVM


/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/svm/_base.py:

Top 1: 0.808338937457969
Logistic regression
Top 1: 0.8090114324142569
Baseline
Top 1: 0.34001344989912574

TypeUse
CatBoost
Top 1: 0.887625
Gradient Boosting
Top 1: 0.876375
Random Forest
Top 1: 0.88525
SVM
Top 1: 0.87325
Linear SVM


/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/nir/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Top 1: 0.869125
Logistic regression
Top 1: 0.877625
Baseline
Top 1: 0.85475



In [112]:
all_usages = []
for target_type in target_types:
    all_usages += list(filter(lambda x: x.annotation_name not in ['java.lang.Override'], usages_by_target_type[target_type]))

test_usages = []
train_usages = []
for usage in all_usages:
    if usage.file_path.startswith('java/idea-ui'):
        test_usages.append(usage)
    else:
        train_usages.append(usage)
train_usages = shuffle(train_usages)

size = 100000

all_usages = train_usages[:size] + test_usages

raw_X = np.array([np.array(usage.features_list, dtype=object) for usage in all_usages])
X = None
all_new_names = []
if len(raw_X) == 0:
    X = np.array([])
    all_new_names = initial_feature_names
else:
    for col in range(raw_X.shape[1]):
        new_columns, new_names = encode_column(raw_X[:, col], size, initial_feature_names[col], 100)
        if new_columns is None:
            continue
        all_new_names += new_names
        if X is None:
            X = new_columns
        else:
            X = np.concatenate((X, new_columns), axis=1)
y = [usage.annotation_name for usage in all_usages]

In [113]:
X_train = []
X_test = []
y_train = []
y_test = []
for i in range(len(X)):
    if all_usages[i].file_path.startswith('java/idea-ui'):
        X_test.append(X[i])
        y_test.append(y[i])
    else:
        X_train.append(X[i])
        y_train.append(y[i])

In [114]:
len(X_test)

3046

In [115]:
len(X_train)

100000

In [35]:
# X_train, y_train = shuffle(X_train, y_train)
# X_train = X_train[:1000]
# y_train = y_train[:1000]

In [116]:
def calc(model):
    model.fit(X_train, y_train)
    predicted = model.predict(X_test)

    metric = Metric(predicted, y_test)
    # print(f'Count: {len(y_test)}')
    for i in range(1, 6):
        print(f'Top {i}: {metric.top_i(i)}')

In [117]:
# print('Linear SVM')
# calc(LinearSVM(c=0.2))
# print('Ridge')
# calc(Ridge())
# print('СatBoost')
# calc(CatBoost())
print('Forest')
calc(RandomForest())
# print("Logistic regression")
# calc(LogisticReg(c=0.2))
# print('SVM')
# calc(SVM(c=0.2))
print('Baseline')
calc(Baseline())

Forest
Top 1: 0.7774130006565988
Top 2: 0.9652002626395273
Top 3: 0.9799737360472751
Top 4: 0.9852265265922522
Top 5: 0.9885095206828628
Baseline
Top 1: 0.6966513460275772
Top 2: 0.9182534471437951
Top 3: 0.9182534471437951
Top 4: 0.9425476034143139
Top 5: 0.9445173998686802


In [15]:
def load_for_target2(target_type, ignored_feature, ignored_annotation=()):
    method_usages = list(
        filter(lambda x: x.annotation_name not in ignored_annotation, usages_by_target_type[target_type]))
    raw_X = np.array([np.array(usage.features_list, dtype=object) for usage in method_usages])
    X = None
    all_new_names = []
    if len(raw_X) == 0:
        X = np.array([])
        all_new_names = initial_feature_names
    else:
        for col in range(raw_X.shape[1]):
            if col == ignored_feature:
                continue
            new_columns, new_names = encode_column(raw_X[:, col], initial_feature_names[col])
            if new_columns is None:
                continue
            all_new_names += new_names
            if X is None:
                X = new_columns
            else:
                X = np.concatenate((X, new_columns), axis=1)
    y = np.array([usage.annotation_name for usage in method_usages])
    return X, y, all_new_names

In [17]:
for i in range(len(initial_feature_names)):
    X, y, feature_names = load_for_target2('Method', i)
    rnd = random.Random(42)
    for _ in range(1):
        size = 10000
        X, y = shuffle(X, y, random_state=42)
        X = X[:size]
        y = y[:size]
        if len(X) == 0:
            continue
        svm = SVM()
        print(initial_feature_names[i])
        print("SVM")
        calculate(X, y, svm)
        print("Baseline")
        calculate(X, y, Baseline())
        print()

0
SVM
Top 1: 0.834
Baseline
Top 1: 0.505125

1
SVM
Top 1: 0.842625
Baseline
Top 1: 0.505125

2
SVM
Top 1: 0.819625
Baseline
Top 1: 0.505125

3
SVM
Top 1: 0.81025
Baseline
Top 1: 0.505125

4
SVM
Top 1: 0.724125
Baseline
Top 1: 0.505125

5
SVM
Top 1: 0.84425
Baseline
Top 1: 0.505125

6
SVM
Top 1: 0.82475
Baseline
Top 1: 0.505125

7
SVM
Top 1: 0.82775
Baseline
Top 1: 0.505125

8
SVM
Top 1: 0.839625
Baseline
Top 1: 0.505125

9
SVM
Top 1: 0.839625
Baseline
Top 1: 0.505125



In [18]:
def load_for_target3(target_type, used_feature, ignored_annotation=()):
    method_usages = list(
        filter(lambda x: x.annotation_name not in ignored_annotation, usages_by_target_type[target_type]))
    raw_X = np.array([np.array(usage.features_list, dtype=object) for usage in method_usages])
    X = None
    all_new_names = []
    if len(raw_X) == 0:
        X = np.array([])
        all_new_names = initial_feature_names
    else:
        for col in range(raw_X.shape[1]):
            if col != used_feature:
                continue
            new_columns, new_names = encode_column(raw_X[:, col], initial_feature_names[col])
            if new_columns is None:
                continue
            all_new_names += new_names
            if X is None:
                X = new_columns
            else:
                X = np.concatenate((X, new_columns), axis=1)
    y = np.array([usage.annotation_name for usage in method_usages])
    return X, y, all_new_names

In [19]:
for i in range(len(initial_feature_names)):
    X, y, feature_names = load_for_target3('Method', i)
    rnd = random.Random(42)
    for _ in range(1):
        size = 10000
        X, y = shuffle(X, y, random_state=42)
        X = X[:size]
        y = y[:size]
        if len(X) == 0:
            continue
        svm = SVM()
        print(initial_feature_names[i])
        print("SVM")
        calculate(X, y, svm)
        print("Baseline")
        calculate(X, y, Baseline())
        print()

0
SVM
Top 1: 0.611125
Baseline
Top 1: 0.505125

1
SVM
Top 1: 0.5655
Baseline
Top 1: 0.505125

2
SVM
Top 1: 0.525
Baseline
Top 1: 0.505125

3
SVM
Top 1: 0.581875
Baseline
Top 1: 0.505125

4
SVM
Top 1: 0.60675
Baseline
Top 1: 0.505125

5
SVM
Top 1: 0.60975
Baseline
Top 1: 0.505125

6
SVM
Top 1: 0.655
Baseline
Top 1: 0.505125

7
SVM
Top 1: 0.514125
Baseline
Top 1: 0.505125

8
SVM
Top 1: 0.505125
Baseline
Top 1: 0.505125

9
SVM
Top 1: 0.505125
Baseline
Top 1: 0.505125

